In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Glioblastoma"
cohort = "GSE175700"

# Input paths
in_trait_dir = "../../input/GEO/Glioblastoma"
in_cohort_dir = "../../input/GEO/Glioblastoma/GSE175700"

# Output paths
out_data_file = "../../output/preprocess/Glioblastoma/GSE175700.csv"
out_gene_data_file = "../../output/preprocess/Glioblastoma/gene_data/GSE175700.csv"
out_clinical_data_file = "../../output/preprocess/Glioblastoma/clinical_data/GSE175700.csv"
json_path = "../../output/preprocess/Glioblastoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Identification of indoleamine 2, 3-dioxygenase 1 (IDO1) regulated genes in human glioblastoma cell line U87"
!Series_summary	"Transcriptome analysis of U87 cells under different treatments to identify IDO1-regulated genes"
!Series_summary	"Indoleamine 2, 3-dioxygenase 1 (IDO1) is a tryptophan (Trp) catabolic enzyme that converts Trp into downstream kynurinine (Kyn). Many studies have indicated that IDO1 is a critical suppressive immune checkpoint molecule invovled in various types of cancer. Canonically, the underlying mechanism of IDO1 immunosuppressive role is related with its enzyme activity, that is the depletion of Trp and accumulation of Kyn lead to increased tumor infiltrating suppressive regulatory T cells. Recent studies, however, challenged this hypothesis and imply that tumor cell-derived IDO1 can mediate immunosuppression independent of its enzyme activity. In this study, we aim to identify genes that are regulated by IDO1 in human gli

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this is a microarray study of gene expression
# in U87 glioblastoma cells under different treatment conditions
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# Looking at the sample characteristics dictionary

# 2.1 Data Availability
# From the sample characteristics dictionary and background information:
# For trait (Glioblastoma): The dataset consists of U87 glioblastoma cells
# Everyone in the dataset has glioblastoma (cell line), so trait is constant
trait_row = None  # Trait data is not useful for association study since it's constant

# For age: No age information provided
age_row = None

# For gender: There's 'Sex: male' at index 1
gender_row = 1

# 2.2 Data Type Conversion functions
def convert_trait(value):
    # Not used since trait_row is None, but defining for completeness
    if value and ":" in value:
        trait_value = value.split(":", 1)[1].strip().lower()
        if "glioblastoma" in trait_value:
            return 1
        else:
            return 0
    return None

def convert_age(value):
    # Not used since age_row is None, but defining for completeness
    if value and ":" in value:
        age_value = value.split(":", 1)[1].strip()
        try:
            return float(age_value)
        except ValueError:
            return None
    return None

def convert_gender(value):
    if value and ":" in value:
        gender_value = value.split(":", 1)[1].strip().lower()
        if "female" in gender_value:
            return 0
        elif "male" in gender_value:
            return 1
        else:
            return None
    return None

# 3. Save Metadata
# Determine trait data availability (if trait_row is not None)
is_trait_available = trait_row is not None

# Use the validate_and_save_cohort_info function for initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (if trait_row is not None)
# In this case, trait_row is None, so we skip this step
# But let's still include the code for completeness with a condition

if trait_row is not None:
    # Define the sample characteristics dictionary
    sample_characteristics = {0: ['tissue: brain'], 1: ['Sex: male']}
    
    # Define the dataframe for clinical data
    clinical_data = pd.DataFrame(list(sample_characteristics.values()), 
                                index=sample_characteristics.keys(),
                                columns=["characteristics"])
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the output
    preview = preview_df(selected_clinical_df)
    print("Clinical data preview:", preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 69
Header line: "ID_REF"	"GSM5344433"	"GSM5344434"	"GSM5344435"	"GSM5344436"	"GSM5344437"	"GSM5344438"	"GSM5344439"	"GSM5344440"	"GSM5344441"	"GSM5344442"	"GSM5344443"	"GSM5344444"	"GSM5344445"	"GSM5344446"	"GSM5344447"	"GSM5344448"	"GSM5344449"	"GSM5344450"	"GSM5344451"	"GSM5344452"	"GSM5344453"	"GSM5344454"	"GSM5344455"	"GSM5344456"	"GSM5344457"	"GSM5344458"	"GSM5344459"	"GSM5344460"	"GSM5344461"	"GSM5344462"	"GSM5344463"	"GSM5344464"	"GSM5344465"	"GSM5344466"	"GSM5344467"	"GSM5344468"
First data line: "AFFX-BkGr-GC03_st"	8.7173	9.07515	8.58114	7.23554	8.72152	8.81188	9.45325	8.28566	10.5923	8.7657	8.08258	8.94281	9.22912	9.4206	10.2476	9.18288	8.11761	8.5086	7.88719	9.10813	8.64127	9.05306	8.84052	7.82312	9.88867	9.80206	10.9257	9.94282	9.51898	8.61312	9.44908	9.06246	8.82998	9.3153	9.54165	9.37893


Index(['AFFX-BkGr-GC03_st', 'AFFX-BkGr-GC04_st', 'AFFX-BkGr-GC05_st',
       'AFFX-BkGr-GC06_st', 'AFFX-BkGr-GC07_st', 'AFFX-BkGr-GC08_st',
       'AFFX-BkGr-GC09_st', 'AFFX-BkGr-GC10_st', 'AFFX-BkGr-GC11_st',
       'AFFX-BkGr-GC12_st', 'AFFX-BkGr-GC13_st', 'AFFX-BkGr-GC14_st',
       'AFFX-BkGr-GC15_st', 'AFFX-BkGr-GC16_st', 'AFFX-BkGr-GC17_st',
       'AFFX-BkGr-GC18_st', 'AFFX-BkGr-GC19_st', 'AFFX-BkGr-GC20_st',
       'AFFX-BkGr-GC21_st', 'AFFX-BkGr-GC22_st'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers in the gene expression data
# The identifiers like 'AFFX-BkGr-GC03_st' are Affymetrix probe IDs, not human gene symbols
# These are microarray probe identifiers that need to be mapped to gene symbols
# The "_st" suffix indicates these are from an Affymetrix GeneChip ST (Sense Target) array

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE175700
Line 6: !Series_title = Identification of indoleamine 2, 3-dioxygenase 1 (IDO1) regulated genes in human glioblastoma cell line U87
Line 7: !Series_geo_accession = GSE175700
Line 8: !Series_status = Public on Dec 10 2021
Line 9: !Series_submission_date = May 27 2021
Line 10: !Series_last_update_date = Dec 11 2021
Line 11: !Series_pubmed_id = 34479957
Line 12: !Series_summary = Transcriptome analysis of U87 cells under different treatments to identify IDO1-regulated genes
Line 13: !Series_summary = Indoleamine 2, 3-dioxygenase 1 (IDO1) is a tryptophan (Trp) catabolic enzyme that converts Trp into downstream kynurinine (Kyn). Many studies have indicated that IDO1 is a critical suppressive immune che


Gene annotation preview:
{'ID': ['TC0100006432.hg.1', 'TC0100006433.hg.1', 'TC0100006434.hg.1', 'TC0100006435.hg.1', 'TC0100006436.hg.1'], 'probeset_id': ['TC0100006432.hg.1', 'TC0100006433.hg.1', 'TC0100006434.hg.1', 'TC0100006435.hg.1', 'TC0100006436.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['11869', '28046', '29554', '52473', '62948'], 'stop': ['14412', '29178', '31109', '53312', '63887'], 'total_probes': [10, 6, 10, 10, 10], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// OTTHUMT00000002844 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// OTTHUMT00000362751 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102', 'spopoybu.aAug10-unspliced // spopoybu // Transcript Identified by AceView // --- // ---', 'NR_036267 // MIR1302-10 // microRNA 1302-10 // --- // 10042

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns in gene_annotation contain probe IDs and gene symbols
# From examining the gene_annotation dataframe:
# 'ID' column contains the probe IDs that match the gene expression data
# 'gene_assignment' column contains gene symbol information, but needs to be processed

# 2. Create a gene mapping dataframe with probe IDs and gene symbols
# First, let's extract the columns and rename gene_assignment to 'Gene' as required by apply_gene_mapping
mapping_data = gene_annotation[['ID', 'gene_assignment']].copy()
mapping_data = mapping_data.rename(columns={'gene_assignment': 'Gene'})

# 3. Apply the mapping to convert probe-level measurements to gene expression
# Use the apply_gene_mapping function, which will:
# - Extract human gene symbols from the gene_assignment text
# - Handle the many-to-many mapping between probes and genes
# - Split probe values proportionally when a probe maps to multiple genes
# - Sum all probe contributions for each gene
gene_data = apply_gene_mapping(gene_data, mapping_data)

# Add validation to check if the mapping was successful
if gene_data.empty:
    print("Warning: No genes were successfully mapped. Check the gene annotation format.")
else:
    # Preview the results
    print("Gene expression data after mapping:")
    print(f"Shape: {gene_data.shape}")
    print(f"First 5 gene symbols: {list(gene_data.index[:5])}")
    print("Sample of gene expression values:")
    print(gene_data.iloc[:5, :5])

    # Normalize gene symbols in the index to handle synonyms and aggregate rows with same symbol
    gene_data = normalize_gene_symbols_in_index(gene_data)
    print("\nAfter normalizing gene symbols:")
    print(f"Shape: {gene_data.shape}")
    print(f"First 5 gene symbols: {list(gene_data.index[:5])}")

    # Save the gene expression data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to {out_gene_data_file}")


Gene expression data after mapping:
Shape: (73312, 36)
First 5 gene symbols: ['A-', 'A-52', 'A-E', 'A-I', 'A-II']
Sample of gene expression values:
      GSM5344433  GSM5344434  GSM5344435  GSM5344436  GSM5344437
Gene                                                            
A-     33.056562   33.313759   33.639137   33.114868   33.792869
A-52   19.138190   18.546140   19.292170   19.040045   19.265255
A-E     0.769540    0.632636    0.748591    0.716990    0.893139
A-I     3.273666    3.064109    3.105160    3.134560    3.060834
A-II    1.306777    1.430533    1.328100    1.298530    1.352080

After normalizing gene symbols:
Shape: (34003, 36)
First 5 gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1']


Gene expression data saved to ../../output/preprocess/Glioblastoma/gene_data/GSE175700.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Load the gene expression data saved in step 6
gene_data = pd.read_csv(out_gene_data_file, index_col=0)
print(f"Loaded gene data shape: {gene_data.shape}")

# Normalize gene symbols using NCBI Gene database (already done in step 6, so we don't need to do it again)
# We'll use the normalized gene data directly from the previous step
normalized_gene_data = gene_data
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"Sample gene symbols after normalization: {list(normalized_gene_data.index[:10])}")

# 2. Create mock clinical data since this dataset doesn't have real clinical features
# Based on the background information, this is a cell line study with no clinical variation
print("\nCreating mock clinical data - this is a cell line experiment with no clinical variation")
# Create a clinical DataFrame with just the trait column (all samples have glioblastoma)
sample_ids = normalized_gene_data.columns
mock_clinical_data = pd.DataFrame(
    {trait: [1] * len(sample_ids)},  # All samples are glioblastoma cell lines
    index=sample_ids
)
print(f"Mock clinical data shape: {mock_clinical_data.shape}")
print("Mock clinical data preview:")
print(mock_clinical_data.head())

# Save the mock clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
mock_clinical_data.to_csv(out_clinical_data_file)

# 3. Link clinical and genetic data
linked_data = pd.concat([mock_clinical_data, normalized_gene_data.T], axis=1)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, first 5 columns):")
if linked_data.shape[1] >= 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data.head())

# 4. Handle missing values
print("\nMissing values before handling:")
print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
gene_cols = [col for col in linked_data.columns if col != trait]
if gene_cols:
    print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
    print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")

cleaned_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {cleaned_data.shape}")

# 5. Evaluate bias in trait and demographic features
is_trait_biased = True  # Set to True since all samples have the same trait value (all are glioblastoma)
print("\nEvaluating trait bias:")
print(f"All samples in this dataset are glioblastoma cell lines under different experimental conditions.")
print(f"Since there is no variation in the trait, this dataset is biased and not suitable for associational studies.")

# 6. Final validation and save
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=len(normalized_gene_data) > 0, 
    is_trait_available=True,  # The trait is available, but lacks variation
    is_biased=is_trait_biased, 
    df=cleaned_data,
    note="Dataset contains gene expression from glioblastoma cell line U87 under different treatments, but lacks trait variation."
)

# 7. Since this is a cell line experiment without clinical variation,
# we won't save the linked data as it's not suitable for trait-gene association studies
if is_usable and len(cleaned_data) > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("This dataset is not usable for trait-gene association studies as it lacks trait variation.")

Loaded gene data shape: (34003, 36)
Normalized gene data shape: (34003, 36)
Sample gene symbols after normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2ML1-AS1', 'A2ML1-AS2', 'A2MP1', 'A3GALT2']

Creating mock clinical data - this is a cell line experiment with no clinical variation
Mock clinical data shape: (36, 1)
Mock clinical data preview:
            Glioblastoma
GSM5344433             1
GSM5344434             1
GSM5344435             1
GSM5344436             1
GSM5344437             1
Linked data shape: (36, 34004)
Linked data preview (first 5 rows, first 5 columns):
            Glioblastoma      A1BG  A1BG-AS1      A1CF       A2M
GSM5344433             1  3.306317  2.422265  0.951533  4.913935
GSM5344434             1  3.350438  2.392850  0.988175  4.899650
GSM5344435             1  3.103491  2.201615  0.952673  4.918110
GSM5344436             1  3.343382  2.464650  0.902613  4.953450
GSM5344437             1  3.484879  2.545615  1.205907  5.221790

Missi

Data shape after handling missing values: (36, 34004)

Evaluating trait bias:
All samples in this dataset are glioblastoma cell lines under different experimental conditions.
Since there is no variation in the trait, this dataset is biased and not suitable for associational studies.
This dataset is not usable for trait-gene association studies as it lacks trait variation.
